In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks, savgol_filter
import re
import csv
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import itertools
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path
from scipy.io import loadmat 
from pathlib import Path

In [7]:
def read_csv_data(main_folder_path):
    data = {}

    # Recorre todas las subcarpetas en el directorio principal (Nivel 1)
    for subfolder1 in os.listdir(main_folder_path):
        subfolder1_path = os.path.join(main_folder_path, subfolder1)

        # Verifica que sea un directorio
        if os.path.isdir(subfolder1_path):
            data[subfolder1] = {}

            # Recorre todas las subcarpetas dentro de la subcarpeta Nivel 1 (Nivel 2)
            for subfolder2 in os.listdir(subfolder1_path):
                subfolder2_path = os.path.join(subfolder1_path, subfolder2)

                # Verifica que sea un directorio
                if os.path.isdir(subfolder2_path):
                    data[subfolder1][subfolder2] = {}

                    # Recorre todas las subcarpetas dentro de la subcarpeta Nivel 2 (Nivel 3)
                    for subfolder3 in os.listdir(subfolder2_path):
                        subfolder3_path = os.path.join(subfolder2_path, subfolder3)

                        # Verifica que sea un directorio
                        if os.path.isdir(subfolder3_path):
                            data[subfolder1][subfolder2][subfolder3] = []

                            # Recorre todos los archivos en la subcarpeta Nivel 3
                            for filename in os.listdir(subfolder3_path):
                                # Solo procesa archivos con extensión .csv
                                if filename.endswith(".mat"):
                                    file_path = os.path.join(subfolder3_path, filename)

                                    # Lee el archivo mat, omitiendo la primera fila, sin encabezados, y separa los datos por comas
                                    df = loadmat(file_path)
                                    df = df["rconst"][0]
                                    df = pd.DataFrame({"I": df.real, "Q":df.imag})
                                    # Añade una tupla (nombre del archivo, DataFrame) a la lista correspondiente
                                    data[subfolder1][subfolder2][subfolder3].append((filename, df))

    return data

# Especifica la ruta a tu carpeta principal
#main_folder_path = "/home/hanna//32GB"
main_folder_path ="/Users/Camilo Anacona/Documents/Proyecto2James"

# Llama a la función y almacena los datos en un diccionario
data = read_csv_data(main_folder_path)


# Opcional: imprimir la cantidad de archivos leídos por subcarpeta
for subfolder1, subfolders2 in data.items():
    for subfolder2, subfolders3 in subfolders2.items():
        for subfolder3, files in subfolders3.items():
            print(f"Subcarpeta '{subfolder1} > {subfolder2} > {subfolder3}': se han leído {len(files)} archivos CSV.")



Subcarpeta '32GBd > 0km_0dBm > 28p5GHz': se han leído 4 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 29GHz': se han leído 4 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 29p5GHz': se han leído 24 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 30GHz': se han leído 44 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 30p5GHz': se han leído 69 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 31GHz': se han leído 66 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 31p5GHz': se han leído 71 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 32GHz': se han leído 74 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 32p5GHz': se han leído 84 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 33GHz': se han leído 98 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 33p5GHz': se han leído 88 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 34GHz': se han leído 104 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 34p5GHz': se han leído 99 archivos CSV.
Subcarpeta '32GBd > 0km_0dBm > 35GHz': se han leído 88 archivos CSV.
Subcarpeta '32GBd > 0

In [8]:
def list_subfolders1(main_folder_path):
    dat = {}

    # Recorre todas las subcarpetas en el directorio principal (Nivel 1)
    for subfolder1 in os.listdir(main_folder_path):
        subfolder1_path = os.path.join(main_folder_path, subfolder1)

        # Verifica que sea un directorio en el Nivel 1
        if os.path.isdir(subfolder1_path):
            # Recorre todas las subcarpetas dentro de la subcarpeta de Nivel 1 (Nivel 2)
            for subfolder2 in os.listdir(subfolder1_path):
                subfolder2_path = os.path.join(subfolder1_path, subfolder2)

                # Verifica que sea un directorio en el Nivel 2
                if os.path.isdir(subfolder2_path):
                    print(f"Subcarpeta en Nivel 2: {subfolder2}")  # Imprime el nombre de la subcarpeta encontrada en el Nivel 2
    
    return dat

#Encontrar picos        
def find_peak_in_segment(counts, bin_edges, start, end):
    start_idx = np.searchsorted(bin_edges, start, side='left')
    end_idx = np.searchsorted(bin_edges, end, side='left')
    
    # Verificar que el segmento no esté vacío
    if start_idx >= end_idx or start_idx >= len(counts):
        print(f"Segmento vacío o fuera de límites: start={start}, end={end}")
        return None  # O algún valor predeterminado, como -1, para indicar un error

    # Encontrar el pico más alto en el segmento
    peak_idx = start_idx + np.argmax(counts[start_idx:end_idx])
    
    return peak_idx

#Encontrar valles
# prueba valles
def find_valley_in_segment(counts, bin_edges, start, end):
    # Encontrar los índices de los bins que corresponden al segmento
    start_idx = np.searchsorted(bin_edges, start, side='right')
    end_idx = np.searchsorted(bin_edges, end, side='left')

    # Encontrar el valle más bajo en el segmento
    valley_idx = start_idx + np.argmin(counts[start_idx:end_idx])

    return valley_idx

In [36]:
def plot_histograms_for_subfolder_all(data, subfolder_name, var_name):
    results_all = {}  # Inicializar siempre el diccionario de resultados
    if subfolder_name not in data:
        print(f"La subcarpeta '{subfolder_name}' no existe en los datos.")
        return results_all  # Retornar diccionario vacío si la subcarpeta no existe

    files = data[subfolder_name]

    # Verifica que los archivos sean una lista de tuplas con dos elementos
    for item in files:
        if len(item) != 2:
            print(f"Error: se esperaba una tupla con 2 elementos, pero se encontró: {item}")
            continue

        filename, df = item  # Asume que cada ítem es una tupla de (filename, DataFrame)
        
        # Obtén los datos del histograma
        counts, bin_edges = np.histogram(df[var_name], bins=100)
        
        # Encontrar el pico más alto en cada segmento
        peaks = [
            find_peak_in_segment(counts, bin_edges, -4, -2),
            find_peak_in_segment(counts, bin_edges, -2, 0),
            find_peak_in_segment(counts, bin_edges, 0, 2),
            find_peak_in_segment(counts, bin_edges, 2, 4)
        ]
        
        # Encontrar el valle más bajo en cada segmento
        valleys = [
            find_valley_in_segment(counts, bin_edges, -3, -1),
            find_valley_in_segment(counts, bin_edges, -1, 1),
            find_valley_in_segment(counts, bin_edges, 1, 3)
        ]
        
        # Filtrar los valores None si no se encontraron picos en algún segmento
        peaks = [peak for peak in peaks if peak is not None]
        valleys = [valley for valley in valleys if valley is not None]
        
        # Guardar los bin_edges de los picos y valles encontrados
        peaks_bin_edges = bin_edges[peaks]
        valleys_bin_edges = bin_edges[valleys]
        
        # Calcular el ensanchamiento
        min_value = min(bin_edges)
        max_value = max(bin_edges)
        ensanchamiento = max_value - min_value
        
        # Guardar los resultados
        results_all[f'{subfolder_name}_{filename}'] = {
            'peaks': peaks_bin_edges,
            'valleys': valleys_bin_edges,
            'ensanchamiento': ensanchamiento
        }
        
        # Generar la gráfica del histograma
        #plt.figure(figsize=(10, 6))
        #plt.hist(df[var_name], bins=100, alpha=0.7, color='blue', edgecolor='black')
        #plt.plot(bin_edges[peaks], counts[peaks], 'ro')  # Resaltar los picos
        #plt.plot(bin_edges[valleys], counts[valleys], 'go')  # Resaltar los valles
        #plt.title(f'Histograma de {var_name} - Subcarpeta: {subfolder_name}, Archivo: {filename}')
        #plt.xlabel(f'Valor de {var_name}')
        #plt.ylabel('Frecuencia')
        #plt.grid(True)
        #plt.show()

    return results_all

In [20]:
print(data.items())

dict_items([('.ipynb_checkpoints', {}), ('3x32Gbaud_16QAM', {'270km_0dBm': {'30GHz': [('Song4_Y_24p93dB_30GHz_270km_0dBm.mat',               I         Q
0      1.310095  3.017568
1      4.268198  0.087642
2      0.104091 -1.738077
3      2.821031  0.348048
4     -0.673230  1.696100
...         ...       ...
39526 -1.915510 -3.167299
39527  3.796958  4.453150
39528  3.369277  3.164143
39529  1.319127 -0.304751
39530 -0.593273  0.051388

[39531 rows x 2 columns]), ('Song1_Y_23p05dB_30GHz_270km_0dBm.mat',               I         Q
0      0.277643 -2.544757
1     -2.767118 -4.284170
2     -3.454462 -1.114674
3     -0.110853  0.277507
4      3.043603 -3.999439
...         ...       ...
39526 -1.111249  1.628992
39527  1.111319  0.795971
39528  1.998631  1.718751
39529  0.837759 -1.798924
39530  4.201500 -4.032074

[39531 rows x 2 columns]), ('Song3_X_25p9dB_30GHz_270km_0dBm.mat',               I         Q
0      2.723133 -1.321994
1     -2.214644 -2.600971
2     -2.332470 -3.101774
3      3

In [37]:
def analyze_data(data):
    pattern = r'(\d+(\.\d+)?)GHz'
    pattern_osnr = r'(\d+(\.\d+)?)(?=dB)'

    matriz_caract = np.empty((0, 18))  # Inicializa la matriz vacía
    count = 0  # Contador para limitar a 3 histogramas

    for subfolder1, subfolders2 in data.items():
        
       
        for subfolder2, subfolders3 in subfolders2.items():
            print(subfolder2)
            if 'dBm' in subfolder2:  # Verificar que el nombre de la subcarpeta contenga 'dBm'
                print(f'se está leyendo: {subfolder2}')
                for subfolder3, files in subfolders3.items(): # <-
                    print(f'Carpeta en Nivel 3: {subfolder3}, Archivos: {files}')
 
                    if 'GHz' in subfolder3:  # Verificar que el nombre de la subcarpeta contenga 'GHz'
                        print(f'se está leyendo: {subfolder3} con GHz')
                        for filename, df in files:
                            match = re.search(pattern, subfolder3)  # Busca en el nombre de la subcarpeta
                            #print(f'hay match')
                            spacing = float(match.group(1)) if match else None

                            # Aquí usamos subfolder3 en lugar de filename
                            resultsI = plot_histograms_for_subfolder_all(data[subfolder1][subfolder2], subfolder3, 'I')
                            #print(f"MUERTEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE :3")
                            resultsQ = plot_histograms_for_subfolder_all(data[subfolder1][subfolder2], subfolder3, 'Q')

                            for resultI, resultQ in zip(resultsI.items(), resultsQ.items()):
                                #if count >= 3:  # Limitar a 3 histogramas
                                 #   print("Se han generado 3 histogramas. Finalizando...")
                                  #  return matriz_caract

                                match = re.search(pattern_osnr, resultI[0])
                                OSNR = float(match.group(1)) if match else None

                                I = resultI[1]
                                Q = resultQ[1]

                                vector = np.hstack((
                                    I["peaks"], Q["peaks"],
                                    I["valleys"], Q["valleys"],
                                    I["ensanchamiento"],
                                    Q["ensanchamiento"],
                                    [spacing, OSNR]
                                ))

                                matriz_caract = np.vstack((matriz_caract, vector))
                                count += 1  # Incrementa el contador
                    else:
                        print("No se encontró la subcarpeta 'GHz'.")



    return matriz_caract

# Especifica la ruta a tu carpeta principal
#main_folder_path = 'Song1_Y_23p05dB_30GHz_270km_0dBm.mat'

# Llama a la función y almacena los datos en un diccionario
#data = read_csv_data(main_folder_path)

# Análisis de datos
matriz_caract = analyze_data(data)

# Aquí puedes realizar más análisis o guardar 'matriz_caract' según lo que necesites

270km_0dBm
se está leyendo: 270km_0dBm
Carpeta en Nivel 3: 30GHz, Archivos: [('Song4_Y_24p93dB_30GHz_270km_0dBm.mat',               I         Q
0      1.310095  3.017568
1      4.268198  0.087642
2      0.104091 -1.738077
3      2.821031  0.348048
4     -0.673230  1.696100
...         ...       ...
39526 -1.915510 -3.167299
39527  3.796958  4.453150
39528  3.369277  3.164143
39529  1.319127 -0.304751
39530 -0.593273  0.051388

[39531 rows x 2 columns]), ('Song1_Y_23p05dB_30GHz_270km_0dBm.mat',               I         Q
0      0.277643 -2.544757
1     -2.767118 -4.284170
2     -3.454462 -1.114674
3     -0.110853  0.277507
4      3.043603 -3.999439
...         ...       ...
39526 -1.111249  1.628992
39527  1.111319  0.795971
39528  1.998631  1.718751
39529  0.837759 -1.798924
39530  4.201500 -4.032074

[39531 rows x 2 columns]), ('Song3_X_25p9dB_30GHz_270km_0dBm.mat',               I         Q
0      2.723133 -1.321994
1     -2.214644 -2.600971
2     -2.332470 -3.101774
3      3.605093 -

Carpeta en Nivel 3: 36GHz, Archivos: [('Song2_Y_24p06dB_36GHz_270km_0dBm.mat',               I         Q
0      3.200038  2.427062
1      2.950168  3.140232
2      2.812331 -3.131212
3     -0.563332  1.311127
4      3.141983 -3.339729
...         ...       ...
39526  2.472724 -0.488929
39527 -2.654552  3.193913
39528 -0.783665 -1.174359
39529  1.053513 -2.810591
39530 -2.897344  2.858699

[39531 rows x 2 columns]), ('Song1_Y_21p96dB_36GHz_270km_0dBm.mat',               I         Q
0     -2.591438  3.376521
1     -2.192708  3.789244
2      3.669869  1.040717
3     -1.140328 -2.499357
4      3.617481  2.228424
...         ...       ...
39526  0.862587  0.615540
39527 -0.925750  1.457237
39528 -0.634395 -2.746916
39529 -3.018347 -0.632669
39530 -0.817913 -1.063707

[39531 rows x 2 columns]), ('Song1_Y_16p02dB_36GHz_270km_0dBm.mat',               I         Q
0     -1.955422 -0.414257
1     -3.033704 -3.432799
2      3.355695 -2.223282
3      0.099945 -3.221510
4     -2.168975 -0.199316
...

Carpeta en Nivel 3: 33p5GHz, Archivos: [('Song3_X_25p04dB_33p5GHz_270km_0dBm.mat',               I         Q
0     -2.448437  1.609796
1      1.824291 -3.539815
2      3.035406  0.970847
3     -3.734777  2.849953
4      1.374062 -2.644111
...         ...       ...
39526 -1.214737 -1.075944
39527 -3.363890 -1.239710
39528  0.581677  0.709837
39529  1.423426  2.994077
39530 -3.564058  0.581544

[39531 rows x 2 columns]), ('Song1_X_22p14dB_33p5GHz_270km_0dBm.mat',               I         Q
0     -3.551949  2.626596
1      2.855673  1.252915
2      1.228454 -3.299947
3     -2.840273  1.428820
4      1.492343  1.429656
...         ...       ...
39526  2.462705 -2.835065
39527  1.564381 -3.249639
39528 -2.535471 -1.581676
39529  2.865594  0.794402
39530  1.803904 -0.930871

[39531 rows x 2 columns]), ('Song4_Y_23p14dB_33p5GHz_270km_0dBm.mat',               I         Q
0      0.437408  0.357706
1     -2.865054 -3.227615
2      3.043889 -1.212880
3      2.945179 -1.555233
4     -0.911729  2.98

Carpeta en Nivel 3: 29p5GHz, Archivos: [('Song4_Y_25p03dB_29p5GHz_270km_0dBm.mat',               I         Q
0     -2.979757  1.575339
1      3.987418  0.991262
2      1.212543  0.499666
3     -3.813300  1.592985
4      2.423213  4.472332
...         ...       ...
39526 -1.124454  3.359009
39527 -2.717188  2.173808
39528  2.026211 -0.836217
39529  0.972930  1.328599
39530  0.148172 -2.410464

[39531 rows x 2 columns])]
se está leyendo: 29p5GHz con GHz
Carpeta en Nivel 3: 32p5GHz, Archivos: [('Song3_Y_24p08dB_32p5GHz_270km_0dBm.mat',               I         Q
0      0.603993 -0.602244
1      2.523996  2.473185
2      1.058993 -2.994753
3      0.236004 -2.285563
4     -2.665997  0.923412
...         ...       ...
39526  0.789192 -1.095650
39527 -0.908907  3.017941
39528 -3.595941 -0.706822
39529 -2.584374 -3.189879
39530 -0.734011  2.734841

[39531 rows x 2 columns]), ('Song2_X_24p08dB_32p5GHz_270km_0dBm.mat',               I         Q
0      4.249479 -1.551377
1     -1.071852  0.902298


Carpeta en Nivel 3: 34GHz, Archivos: [('Song1_X_24p13dB_34GHz_270km_0dBm.mat',               I         Q
0     -0.564087 -1.667256
1     -2.188816  2.217649
2     -0.434493 -0.938612
3     -1.076246  1.574544
4      3.296045 -0.358597
...         ...       ...
39526 -2.678529  1.759166
39527 -0.901278 -3.582369
39528  1.413082  3.118422
39529 -3.042036  2.520621
39530 -3.278271 -1.208937

[39531 rows x 2 columns]), ('Song3_X_20p05dB_34GHz_270km_0dBm.mat',               I         Q
0      1.404140  0.898619
1      0.662739 -0.157748
2     -2.980548  1.506352
3     -0.791555  2.630074
4     -0.819815  0.992999
...         ...       ...
39526  2.956811  1.284410
39527  2.350225  3.443552
39528  0.861760  1.463041
39529 -0.772316  2.379748
39530  0.442502 -0.050948

[39531 rows x 2 columns]), ('Song4_X_22p02dB_34GHz_270km_0dBm.mat',               I         Q
0     -3.323609 -3.054455
1      3.050262 -3.343342
2      2.596473 -1.712259
3     -1.137668  1.098970
4      2.849152 -3.433491
...

...

Carpeta en Nivel 3: 29GHz, Archivos: [('Song1_Y_31p16dB_29GHz_270km_9dBm.mat',               I         Q
0     -1.766323 -0.965605
1     -1.333523  0.873606
2     -3.644046 -0.988534
3     -3.766849 -1.431748
4      1.024699 -1.571324
...         ...       ...
39526  1.247987 -2.186958
39527 -1.077383  3.144700
39528 -1.367932  2.271762
39529 -3.697443 -0.895050
39530 -0.869235  1.211910

[39531 rows x 2 columns]), ('Song2_X_24p03dB_29GHz_270km_9dBm.mat',               I         Q
0      0.649286  1.141944
1      0.972082  0.927999
2     -1.625454 -1.057458
3     -4.078636 -2.873109
4     -3.327498 -1.537828
...         ...       ...
39526  3.597942 -1.196825
39527  1.622532  1.756139
39528 -0.867217  3.646544
39529 -1.221037 -3.739304
39530 -3.876028 -1.620163

[39531 rows x 2 columns]), ('Song1_X_30p02dB_29GHz_270km_9dBm.mat',               I         Q
0     -1.330984  2.342327
1     -1.255740 -0.359681
2     -2.741988 -0.967577
3      0.311472  0.661272
4      2.200396  1.492346
...

Carpeta en Nivel 3: 36GHz, Archivos: [('Song1_Y_20p05dB_36GHz_270km_9dBm.mat',               I         Q
0      0.487194 -2.946049
1     -1.852519 -0.795788
2      0.926078  1.256259
3      0.597216 -3.508737
4      3.392439 -3.327624
...         ...       ...
39526 -0.907877 -0.580546
39527  0.984689  2.346846
39528  3.247036  0.714646
39529 -0.182377  0.280178
39530  1.376443 -2.686703

[39531 rows x 2 columns]), ('Song2_X_30p19dB_36GHz_270km_9dBm.mat',               I         Q
0     -0.954657  1.161304
1     -0.848476  3.081321
2     -0.092366  1.393279
3     -2.977920  2.757893
4     -1.577997 -0.256421
...         ...       ...
39526 -1.375672 -3.211555
39527 -1.015480  3.347586
39528  3.177811  0.920813
39529  1.958337 -2.638023
39530  1.425619  0.721669

[39531 rows x 2 columns]), ('Song1_X_24p09dB_36GHz_270km_9dBm.mat',               I         Q
0      0.484175  3.273610
1      1.438669 -1.227321
2     -2.744132 -2.974544
3     -3.022381  0.816443
4     -1.162197  2.918386
...

Carpeta en Nivel 3: 33p5GHz, Archivos: [('Song4_X_20dB_33p5GHz_270km_9dBm.mat',               I         Q
0     -2.878972 -0.861055
1      0.791177 -3.385336
2      1.413252 -2.389840
3     -0.806408 -3.520870
4     -1.693160  1.370629
...         ...       ...
39526  0.941810 -1.511350
39527  1.868134 -0.305241
39528  2.809896  2.612061
39529  3.062627  0.672004
39530  2.814699  1.398914

[39531 rows x 2 columns]), ('Song3_X_23p05dB_33p5GHz_270km_9dBm.mat',               I         Q
0     -1.218395 -0.727831
1     -3.404759 -1.070026
2      2.229650  1.938669
3      0.851349  2.952552
4     -3.549156  0.746975
...         ...       ...
39526  2.700386 -0.544465
39527  2.706560  2.587560
39528 -0.405008 -0.988976
39529  0.528671 -1.650485
39530 -3.015713  1.080974

[39531 rows x 2 columns]), ('Song1_Y_28p03dB_33p5GHz_270km_9dBm.mat',               I         Q
0     -3.464080 -1.165398
1      0.648645  1.239752
2      1.180915  0.961586
3     -2.842868  0.451262
4     -3.354836  3.13342

Carpeta en Nivel 3: 29p5GHz, Archivos: [('Song1_X_26p06dB_29p5GHz_270km_9dBm.mat',               I         Q
0      3.901034 -1.660843
1     -1.053563  2.556224
2      1.105801  0.900419
3      0.524760  3.821337
4     -0.117432 -2.568742
...         ...       ...
39526 -0.142213  0.182397
39527 -0.133661 -1.420503
39528  0.213724  3.100684
39529 -2.221647  3.686801
39530 -3.622810 -0.156598

[39531 rows x 2 columns]), ('Song3_X_26p06dB_29p5GHz_270km_9dBm.mat',               I         Q
0      1.211778 -0.115694
1      1.377855  1.981865
2     -4.115824 -3.093734
3     -3.050590  3.228038
4     -0.776227  2.913026
...         ...       ...
39526 -4.099105 -0.268998
39527 -1.408105  0.016221
39528 -0.826432  1.592952
39529  0.509301  1.472580
39530  1.550933  2.295268

[39531 rows x 2 columns]), ('Song3_X_29p1dB_29p5GHz_270km_9dBm.mat',               I         Q
0      1.143853 -1.697797
1      1.891331 -1.846786
2      2.784165  3.440159
3      2.124611  1.924595
4      3.779680  3.115

KeyboardInterrupt: 

In [32]:
matriz_caract.shape

(3, 18)

In [ ]:
print(data)

In [2]:
METADATA_PATTERN = re.compile(
    r"Song\d+_[XY]_"                     # Match the prefix (e.g., Song1_X or Song1_Y)
    r"(?P<OSNR>[\d]+(?:p[\d]+)?)dB_"     # Match OSNR (integer or decimal with 'p')
    r"(?P<Spacing>[\d]+(?:p[\d]+)?)GHz_" # Match Spacing (integer or decimal with 'p')
    r"(?P<Distance>[\d]+(?:p[\d]+)?)km_" # Match Distance (integer or decimal with 'p')
    r"(?P<Power>[\d]+(?:p[\d]+)?)dBm"    # Match Power (integer or decimal with 'p')
)

main_folder_path = "3x32Gbaud_16QAM"

In [3]:
dfs = []
sub_folder = Path(Path(main_folder_path) / "3x32Gbaud_16QAM/")
for directory in Path(sub_folder).iterdir():
    if not directory.is_dir():
        continue
    # Iterar todos los archivos con la extensión .mat
    for file_path in directory.rglob("*.csv"):
        match = METADATA_PATTERN.search(file_path.name)
        if not match:
            raise ValueError(
                f"File name {file_path.name} does not match the expected pattern."
            )
    
        # Extraer la metadata
        metadata = {
            key: float(value.replace("p", ".")) for key, value in match.groupdict().items()
        }
        mat = pd.read_csv(file_path, usecols=[0,1], dtype={"I": np.float32, "Q": np.float32})
    
        df = pd.DataFrame(
            {
                "I": mat['I'],
                "Q": mat['Q'],
                "Distance": metadata["Distance"],
                "Power": metadata["Power"],
                "Spacing": metadata["Spacing"],
                "OSNR": metadata["OSNR"],
            }
        )
        dfs.append(df)

# Combinar todos los dataframes en uno solo
df_32gbd = pd.concat(dfs, ignore_index=True)

In [5]:
print(df_32gbd)

                  I         Q  Distance  Power  Spacing   OSNR
0         -3.620020  3.221106       0.0    0.0     28.5  28.11
1          3.886321  1.629083       0.0    0.0     28.5  28.11
2          3.715104  3.970738       0.0    0.0     28.5  28.11
3         -0.802521 -4.227191       0.0    0.0     28.5  28.11
4          2.600286 -0.549269       0.0    0.0     28.5  28.11
...             ...       ...       ...    ...      ...    ...
122941405  2.798427 -2.445010     270.0    9.0     50.0  30.02
122941406  1.600604 -0.896461     270.0    9.0     50.0  30.02
122941407 -3.298663 -3.152226     270.0    9.0     50.0  30.02
122941408  2.720725  0.584847     270.0    9.0     50.0  30.02
122941409  2.659247  3.203851     270.0    9.0     50.0  30.02

[122941410 rows x 6 columns]


In [6]:
distances = [0, 270]
powers = [0, 9]
spacings = np.arange(28.5, 36.5, 0.5)
spacings = np.append(spacings, [37.5, 50])

In [9]:
#Encontrar picos        
def find_peak_in_segment(counts, bin_edges, start, end):
    start_idx = np.searchsorted(bin_edges, start, side='left')
    end_idx = np.searchsorted(bin_edges, end, side='left')
    
    # Verificar que el segmento no esté vacío
    if start_idx >= end_idx or start_idx >= len(counts):
        print(f"Segmento vacío o fuera de límites: start={start}, end={end}")
        return None  # O algún valor predeterminado, como -1, para indicar un error

    # Encontrar el pico más alto en el segmento
    peak_idx = start_idx + np.argmax(counts[start_idx:end_idx])
    
    return peak_idx

#Encontrar valles
# prueba valles
def find_valley_in_segment(counts, bin_edges, start, end):
    # Encontrar los índices de los bins que corresponden al segmento
    start_idx = np.searchsorted(bin_edges, start, side='right')
    end_idx = np.searchsorted(bin_edges, end, side='left')

    # Encontrar el valle más bajo en el segmento
    valley_idx = start_idx + np.argmin(counts[start_idx:end_idx])

    return valley_idx





matriz_carac = []

for distance in distances:
    for power in powers:
        for spacing in spacings:            
            filtered_df = df.loc[(np.isclose(df["Distance"], distance)) & (np.isclose(df["Power"], power)) & (np.isclose(df["Spacing"], spacing))]
            OSNRS = set(filtered_df["OSNR"])
            
            for OSNR in OSNRS:
                osnr_filtered = filtered_df.loc[(np.isclose(df["OSNR"], OSNR))]
                for x in ["I","Q"]:
                    fd = filtered_df[x]
                    
                # Obtén los datos del histograma
                counts, bin_edges = np.histogram(bins=100)

                # Encontrar el pico más alto en cada segmento
                peaks = [
                    find_peak_in_segment(counts, bin_edges, -4, -2),
                    find_peak_in_segment(counts, bin_edges, -2, 0),
                    find_peak_in_segment(counts, bin_edges, 0, 2),
                    find_peak_in_segment(counts, bin_edges, 2, 4)
                ]

                # Encontrar el valle más bajo en cada segmento
                valleys = [
                    find_valley_in_segment(counts, bin_edges, -3, -1),
                    find_valley_in_segment(counts, bin_edges, -1, 1),
                    find_valley_in_segment(counts, bin_edges, 1, 3)
                ]

                # Filtrar los valores None si no se encontraron picos en algún segmento
                peaks = [peak for peak in peaks if peak is not None]
                valleys = [valley for valley in valleys if valley is not None]

                # Guardar los bin_edges de los picos y valles encontrados
                peaks_bin_edges = bin_edges[peaks]
                valleys_bin_edges = bin_edges[valleys]

                # Calcular el ensanchamiento
                min_value = min(bin_edges)
                max_value = max(bin_edges)
                ensanchamiento = [max_value - min_value]
                
                
                
                matriz_carac.append(np.concat((peaks, valleys, ensanchamiento, [OSNR])))
                



TypeError: histogram() missing 1 required positional argument: 'a'